In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from GptCOTPrompter import *
from GptCOTPrompter_BeamSeach import *
from AutoReasoner import *
import dotenv

config = dotenv.dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY']

dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')
# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/training.tsv', sep='\t')

In [5]:
import fasttext
ft = fasttext.load_model('cc.en.300.bin')

In [29]:

# change the majority vote to max score

NNDemo = False
max_demo = 5
template = 'original-sql-py'
# template = 'formatv1-sql-py'
def parallel_codex_func_formatv1(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_BeamSeach(
                                              f'prompt_template/{template}.json',
                                              dataset.iloc[i]['id'], 
                                              dataset.iloc[i]['utterance'], 
                                              dataset.iloc[i]['context'], 
                                              dataset.iloc[i]['targetValue'],  
                                              base_path='./dataset/WikiTableQuestions/',
                                              demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                             )
            codex_prompter.max_demo = max_demo
            # codex_prompter._gen_gpt_prompt()
            codex_prompter._gen_gpt_prompt(NNDemo, ft)
            codex_prompter._get_gpt_prediction(beam_size=2)
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            if "model's maximum context length" in str(e):
                return log
            max_retry -= 1
    return log
    
for program in [ 'sql-py']:
    n_threads = 10
    maxLimit = float('inf')
    # maxLimit = 100
    from joblib import Parallel, delayed
    output_result_file = f'./dataset/WikiTableQuestions/results/CodexAnswerCOTExecutor_BeamSeach_{template}_{program}_NNDemo={NNDemo}_results_pristine-unseen-tables_trial.json'
    logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func_formatv1)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))
    json.dump(logs, open(output_result_file, 'w'), indent=4)
    # evaluate: 
    os.system(f'cd ./dataset/WikiTableQuestions/ && python2 evaluator.py ./results/{output_result_file.split("/")[-1]} && cd ..')

  1%|███▍                                                                                                                                                                                                                                                                                                         | 50/4344 [00:36<1:00:28,  1.18it/s]/mnt/GPT-TabQA/GptCOTPrompter.py:47: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  renewed_df = ps.sqldf(code)
/mnt/GPT-TabQA/GptCOTPrompter.py:47: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  renewed_df = ps.sqldf(code)
/mnt/GPT-TabQA/GptCOTPrompter.py:47: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  renewed_df = ps.sqldf(code)
/mnt/GPT-TabQA/GptCOTPrompter.py:47: UserWarning: the 'timedelta' type is not supported, an

99
99


 10%|█████████████████████████████                                                                                                                                                                                                                                                                               | 420/4344 [08:28<1:40:09,  1.53s/it]

99
99


 25%|███████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                               | 1100/4344 [23:03<1:03:39,  1.18s/it]

...

 26%|█████████████████████████████████████████████████████████████████████████████                                                                                                                                                                                                                              | 1120/4344 [23:33<1:13:00,  1.36s/it]

Counter({'Clay': 8, 'Hard': 2})


 31%|███████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                                                                                                               | 1330/4344 [28:46<1:30:25,  1.80s/it]Process Process-12550:
  File "/usr/local/lib/python3.8/dist-packages/openai/api_requestor.py", line 292, in request_raw
    result = _thread_context.session.request(
Process Process-12554:
Process Process-12518:
Process Process-12553:
Process Process-12543:
Process Process-12538:
Process Process-12551:
Process Process-12547:
Process Process-12552:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (m

KeyboardInterrupt: 

  File "/usr/local/lib/python3.8/dist-packages/openai/api_resources/abstract/engine_api_resource.py", line 100, in create
    response, _, api_key = requestor.request(
  File "/mnt/GPT-TabQA/GptConnector.py", line 27, in gpt
    output = openai.Completion.create(
  File "/usr/local/lib/python3.8/dist-packages/openai/api_resources/abstract/engine_api_resource.py", line 100, in create
    response, _, api_key = requestor.request(
  File "/usr/local/lib/python3.8/dist-packages/openai/api_resources/completion.py", line 31, in create
    return super().create(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/openai/api_requestor.py", line 113, in request
    result = self.request_raw(
  File "/mnt/GPT-TabQA/GptConnector.py", line 27, in gpt
    output = openai.Completion.create(
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/openai/api_requestor.py", line 113,

In [24]:
NNDemo = False
max_demo = 5
template = 'original-sql-py'
program = 'sql-py'
def func(i):
    codex_prompter = CodexAnswerCOTExecutor_BeamSeach(
                                              f'prompt_template/{template}.json',
                                              dataset.iloc[i]['id'], 
                                              dataset.iloc[i]['utterance'], 
                                              dataset.iloc[i]['context'], 
                                              dataset.iloc[i]['targetValue'],  
                                              base_path='./dataset/WikiTableQuestions/',
                                              demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                             )
    codex_prompter.max_demo = max_demo
    # codex_prompter._gen_gpt_prompt()
    codex_prompter._gen_gpt_prompt(NNDemo, ft)
    codex_prompter._get_gpt_prediction(beam_size=2)
    log = codex_prompter._log_dict()
    return log
a = func(35)

In [25]:
print(a)

{'id': 'nu-35', 'utterance': 'how many candidates belong to a party other than republican or democrat?', 'source_csv': './dataset/WikiTableQuestions/csv/203-csv/432.csv', 'target_value': '0', 'predicted_value': '0', 'prompt': 'The database table DF is shown as follows:\n[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss\n---\n[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18\n[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20\n[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14\n...\n[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108\n[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

In [50]:
print(a['decision_prompts'][1])
# print(a['action_prompts'][0])

# print(a['gpt_original_output'][0])

The database table DF is shown as follows:
[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss
---
[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18
[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20
[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14
...
[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108
[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or 

In [57]:
print(a['prompt'])

The database table DF is shown as follows:
[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss
---
[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18
[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20
[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14
...
[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108
[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or 

In [51]:
print(a['prompt'])

The database table DF is shown as follows:
[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss
---
[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18
[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20
[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14
...
[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108
[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or 